# All necessary imports

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from source.code.utils import save_obj
from source.code.utils import load_obj

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas_profiling

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 30000)
pd.set_option('display.max_columns', 30000)

In [ ]:
data_path = '../data/dataset/original/{}.csv'
processed_data_path = '../data/dataset/processed/{}.csv'
profiling_path = '../data/dataset/processed/data_profiling/{}.html'
meta_path = '../data/dataset/processed/meta-info/{}.pkl'

In [ ]:
QUERY_PATTERN = 'n_missing <= 0 & type == \'{}\''

In [ ]:
features_types = ['num_features', 'cat_features', 'bool_features']

# Data reading

Firstly we just load all data into memory, then profile each dataset,

then try to filter features that are most interesting for us at the moment

(continuous, categorical, binary features without na, features with low na percentage etc.).

In [ ]:
dataset_names = ['application_train', 'application_test', 'bureau', 'bureau_balance', 'credit_card_balance', 'installments_payments', 'POS_CASH_balance', 'previous_application', 'sample_submission']

In [ ]:
data_dict = dict(zip(dataset_names, list(map(lambda name: pd.read_csv(filepath_or_buffer=data_path.format(name)), tqdm(dataset_names)))))

In [ ]:
columns_description = pd.read_csv(filepath_or_buffer='../data/dataset/original/HomeCredit_columns_description.csv', encoding='ISO-8859-1', index_col=0)

# Feature description

Here at this picture the general data structure is reflected.

Lots of connections and, as a consequence, lots of hypothetial issues with data.

![Image of data scheme](https://storage.googleapis.com/kaggle-media/competitions/home-credit/home_credit.png)

# General data description

- **application_{train|test}.csv**

This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
Static data for all applications. One row represents one loan in our data sample.

- **bureau.csv**

All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.

- **bureau_balance.csv**

Monthly balances of previous credits in Credit Bureau.
This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.

- **POS_CASH_balance.csv**

Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.

- **credit_card_balance.csv**

Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

- **previous_application.csv**

All previous applications for Home Credit loans of clients who have loans in our sample.
There is one row for each previous application related to loans in our data sample.

- **installments_payments.csv**

Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
There is a) one row for every payment that was made plus b) one row each for missed payment.
One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.

- **HomeCredit_columns_description.csv**

This file contains descriptions for the columns in the various data files.

# Data examples

In [ ]:
name_number = 0

In [ ]:
data_dict[dataset_names[name_number]].head().T # application_train

In [ ]:
name_number = 1

In [ ]:
data_dict[dataset_names[name_number]].head().T # application_test

In [ ]:
name_number = 2

In [ ]:
data_dict[dataset_names[name_number]].head().T # bureau

In [ ]:
name_number = 3

In [ ]:
data_dict[dataset_names[name_number]].head().T # bureau_balance

In [ ]:
name_number = 4

In [ ]:
data_dict[dataset_names[name_number]].head().T # credit_card_balance

In [ ]:
name_number = 5

In [ ]:
data_dict[dataset_names[name_number]].head().T # installments_payments

In [ ]:
name_number = 6

In [ ]:
data_dict[dataset_names[name_number]].head().T # POS_CASH_balance

In [ ]:
name_number = 7

In [ ]:
data_dict[dataset_names[name_number]].head().T # previous_application

In [ ]:
name_number = 8

In [ ]:
data_dict[dataset_names[name_number]].head().T # sample_submission

# Replace 'Y' and 'N' with 1 and 0

# Pandas profiling

Usually this reports are being displayed in notebooks but since train and test have so many variables

it is more convnient to work with report as a Pandas DataFrame which contains meta-information about dataset columns

(number of continuous, categorical, binary columns, number of highly correlated columns etc.).

In [ ]:
profiles_dict = {k: pandas_profiling.ProfileReport(v) for k, v in tqdm(data_dict.items())}

# Store the output

We will store all these reports in case.

Also it would be reasonable to store all meta-information to be able not to recalculate it each time.

# Store visual (html) reports

In [ ]:
_ = list(map(lambda name: profiles_dict[name].to_file(outputfile=profiling_path.format(name)), tqdm(dataset_names)))

# Store reports with feature meta-information

In [ ]:
_ = list(map(lambda name: save_obj(profiles_dict[name].description_set['variables'], meta_path.format(name)), tqdm(dataset_names)))

# Explore variables

Here we just take a look at keys of profiling report dictionary for train (yes, dictionaries, dictionaries and, once again, dictionaries)))

In [ ]:
profiles_dict[dataset_names[0]].description_set.keys()

In [ ]:
pd.DataFrame(data=profiles_dict[dataset_names[0]].description_set['variables'].columns.values, columns=['PROFILING_VARIABLES'])

As you can see the report contains a value with the 'variables' key.

The value is a Pandas DataFrame which contains informations about each column of the dataset.

Each column is described with features displayed above.

Let's take a closer look.

# TRAIN

In [ ]:
name = 0

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

You have probably guessed that the whole count of columns is the sum of count of columns with 'CORR' type and count of columns with some number of missing values.

# TEST

In [ ]:
name = 1

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

Again, this time the whole count of columns is 1 columns less because test dataset does not contain TARGET column.

# BUREAU

In [ ]:
name = 2

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

# BUREAU BALANCE

In [ ]:
name = 3

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

# CREDIT CARD BALANCE

In [ ]:
name = 4

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

# INSTALLMENTS PAYMENTS

In [ ]:
name = 5

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

# POS CASH BALANCE

In [ ]:
name = 6

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

# PREVIOUS APPLICATIONS

In [ ]:
name = 7

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['type'].value_counts()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['n_missing'].value_counts().sum()

In [ ]:
profiles_dict[dataset_names[name]].description_set['variables']['p_missing'].value_counts()

In [ ]:
len(profiles_dict[dataset_names[name]].description_set['variables'])

So far we can say that every dataset has some problems with data quality.

It si possble to highlight two major issues at the moment:
- NAs;
- high corelation.

Perhaps (even very likely) there are other issues but that is not clear so far.

Now to simplify the process we will firstly take those features that do not contain any NaN values and are not highly correlated with other features:

In [ ]:
datasets_num_features = dict(zip(dataset_names, list(map(lambda name: profiles_dict[name].description_set['variables'].query(QUERY_PATTERN.format('NUM'))['type'].index.values, dataset_names))))

In [ ]:
datasets_cat_features = dict(zip(dataset_names, list(map(lambda name: profiles_dict[name].description_set['variables'].query(QUERY_PATTERN.format('CAT'))['type'].index.values, dataset_names))))

In [ ]:
datasets_bin_features = dict(zip(dataset_names, list(map(lambda name: profiles_dict[name].description_set['variables'].query(QUERY_PATTERN.format('BOOL'))['type'].index.values, dataset_names))))

In [ ]:
save_obj(datasets_num_features, meta_path.format('datasets_num_features'))

In [ ]:
save_obj(datasets_cat_features, meta_path.format('datasets_cat_features'))

In [ ]:
save_obj(datasets_bin_features, meta_path.format('datasets_bin_features'))